In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

df = pd.read_json('../../data/ld/prokanon.ld_combs4.json')
df['count'] = df['count'].apply(lambda x: x if x >=300 else random.randint(1, 300))
df.sort_values(by='count',inplace=True)


df['cumsum'] = df.groupby('country')['count'].cumsum()

sum_per_country = df.groupby('country')['count'].sum()

df['cdf'] = df['cumsum'] / df['country'].map(sum_per_country)


paises = df[['country']].drop_duplicates()


In [2]:
df

,_id,age_range,seniority,country,gender,count,cumsum,cdf
2710,{'$oid': '65b0ed1817f3c67cf286a581'},"(55,2147483647)",9,Liechtenstein,MALE,1,1,5.134525e-05
10783,{'$oid': '65b7713160aac84946a72da4'},"(55,2147483647)",8,Fiji,FEMALE,1,1,1.352887e-05
3825,{'$oid': '65b0ef9e17f3c67cf286b41e'},"(18,24)",9,Romania,FEMALE,1,1,6.597938e-07
12060,{'$oid': '65b773e760aac84946a74d3f'},"(35,54)",8,Tonga,MALE,1,1,7.081150e-05
10852,{'$oid': '65b7715360aac84946a72f46'},"(35,54)",7,French Polynesia,MALE,1,1,3.081094e-05
...,...,...,...,...,...,...,...,...
187,{'$oid': '65b0e6c717f3c67cf2866927'},"(25,34)",4,United States,FEMALE,7200000,101142200,7.451051e-01
181,{'$oid': '65b0e6c317f3c67cf2866900'},"(35,54)",3,United States,FEMALE,7600000,108742200,8.010935e-01
5298,{'$oid': '65b0f30417f3c67cf286c777'},"(25,34)",3,India,MALE,8500000,49163180,1.000000e+00
178,{'$oid': '65b0e6c117f3c67cf28668ea'},"(25,34)",3,United States,MALE,13000000,121742200,8.968633e-01


In [3]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = res_geq090.groupby('country')['cdf'].apply(lambda x: (x - 0.1).abs().idxmin())

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = res_geq095.groupby('country')['cdf'].apply(lambda x: (x - 0.05).abs().idxmin())

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = res_geq099.groupby('country')['cdf'].apply(lambda x: (x - 0.01).abs().idxmin())

results_99 = res_geq099.loc[idx_min]

In [4]:



for index, row in results_90[['country','count']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_90'] = row['count']


for index, row in results_95[['country','count']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_95'] = row['count']


for index, row in results_99[['country','count']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_99'] = row['count']



for index, value in sum_per_country.items():
    mask = paises['country'] == index
    paises.loc[mask,'RK^4_90'] = (paises.loc[mask,'K^4_90']/value) * 100 
    paises.loc[mask,'RK^4_95'] = (paises.loc[mask,'K^4_95']/value) * 100 
    paises.loc[mask,'RK^4_99'] = (paises.loc[mask,'K^4_99']/value) * 100 

print(len(df.index))

16160


In [5]:
nan_rows = paises[paises.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [country, K^4_90, K^4_95, K^4_99, RK^4_90, RK^4_95, RK^4_99]
Index: []


In [6]:
paises.dropna(inplace=True)
paises['K^4_90'] = paises['K^4_90'].astype('int')
paises['K^4_95'] = paises['K^4_95'].astype('int')
paises['K^4_99'] = paises['K^4_99'].astype('int')
paises['RK^4_90'] = paises['RK^4_90'].round(3)
paises['RK^4_95'] = paises['RK^4_95'].round(3)
paises['RK^4_99'] = paises['RK^4_99'].round(3)


paises['country'] = paises['country'].str.strip()  
paises['country'] = paises['country'].replace({'Turkey': 'Türkiye'})
paises['country'] = paises['country'].apply(lambda x: pycountry.countries.lookup(x).alpha_2)

paises.to_parquet('../../data/analyzed/ld/LD_K^4_X_by_country.parquet')
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
2710,LI,126,103,58,0.647,0.529,0.298
10783,FJ,440,178,78,0.595,0.241,0.106
3825,RO,7800,4400,1700,0.515,0.290,0.112
12060,TO,122,79,46,0.864,0.559,0.326
10852,PF,180,129,57,0.555,0.397,0.176
...,...,...,...,...,...,...,...
3374,NL,32000,20000,7700,0.492,0.307,0.118
10575,AU,55000,27000,10000,0.614,0.302,0.112
95,CA,90000,38000,16000,0.651,0.275,0.116
3615,GB,130000,80000,27000,0.544,0.335,0.113
